In [ ]:
%reload_ext autoreload
%autoreload 2

from dataset.graph_builder import GraphBuilder

builder = GraphBuilder(dataset_path="data/MOLWENI/test.json", dataset_name="MOLWENI")
builder()

In [ ]:
builder.display_graph(0)